In [33]:
import os
import h5py
import numpy as np
import pandas as pd
from data_helpers import grouper
from tempConv import tempConvDecoder
from sklearn.metrics import confusion_matrix

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
## get and format data
# lfp power bands
neural_data_file = h5py.File('datasets/GRat31_636061_all_sorted_spikes.hdf5', 'r')
neural_data = np.asarray(neural_data_file['sorted_spikes']) # iterate through powerbands
neural_data.shape

(80690, 34)

In [3]:
head_signals_h5 = h5py.File('datasets/GRat31_636061_all_head_data.hdf5', 'r')
idx_start, idx_stop = [3,6]

# dx
head_signal = np.asarray(
    [np.asarray(head_signals_h5[key]) for key in head_signals_h5.keys()][0:9]
).T[:,idx_start:idx_stop]
print('head_signals shape: ', head_signal.shape)

# head_signals_keys = list(head_signals_h5.keys())[0:9][idx_start:idx_stop]
# head_signals_int = ['yaw_abs', 'roll_abs', 'pitch_abs']
# print('head_signals_keys intuitive: ', head_signals_int)

head_signals shape:  (80690, 3)


In [58]:
# try taking absolute magnitude of all dx, dy, dz
y = np.sqrt(np.square(head_signal[:,0])+np.square(head_signal[:,1])+np.square(head_signal[:,2]))
y[:10]

array([ 0.05430986,  0.01605095,  0.01133895,  0.01390388,  0.00760131,
        0.02972528,  0.01865461,  0.01811274,  0.02700008,  0.01893541])

In [6]:
# make head signals binary
head_signal = np.sign(head_signal)
head_signal[head_signal<0] = 0
print('head_signals shape: ', head_signal.shape)

head_signals shape:  (80690,)


In [18]:
# make head signals thresholded binary for movement or no movement
head_signal[head_signal>0.1] = 1
head_signal[head_signal<-0.1] = 1
head_signal[head_signal<0.1] = 0
print('head_signals shape: ', head_signal.shape)

head_signals shape:  (80690,)


In [59]:
y[y>0.1] = 1
y[y<0.1] = 0
print('head_signals shape: ', y.shape)
np.histogram(y)

head_signals shape:  (80690,)


(array([49804,     0,     0,     0,     0,     0,     0,     0,     0, 30886]),
 array([ 0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ]))

In [61]:
1-30/(50+30)

0.625

In [7]:
stats = []

In [63]:
TCD = tempConvDecoder(neural_data,y,['yaw_abs'],window=60, offset=30, nb_filter=10, eps=1, regressor=False, lr=0.000001)
TCD.fit()
R2s,rs = TCD.determine_fit()
stats.append([R2s, rs])

X_train and y_train shape: (40233, 60, 34) (40233, 1)
Train on 40233 samples, validate on 40233 samples
Epoch 1/1
40233/40233 [==============================] - 1s - loss: 0.5430 - binary_accuracy: 0.7229 - val_loss: 0.5216 - val_binary_accuracy: 0.7699

In [49]:
print(stats)

[[[0.031335573358747926], [0.44710916743939016]], [[0.030854799427534996], [0.41781672502187289]], [[0.022740946245136562], [0.40926178580167749]], [[0.15305565370523522], [0.47199295774758659]], [[0.29723049771566235], [0.56423019434111854]], [[0.36563370169367715], [0.60758494005721786]], [[0.24284237956721089], [0.52159547396710604]], [[0.24469942913792631], [0.50982654898191426]], [[0.23564322397730364], [0.5131946820165838]], [[0.26918824957093657], [0.53728062133379872]], [[0.28206590787106722], [0.53406484553512557]], [[0.1952049762957615], [0.49160434574808548]]]


In [65]:
np.histogram(TCD.y_test)
15680/24553

0.6386184987577893

In [43]:
# confusion_matrix(TCD.y_test,TCD.y_test_hat)
# list(zip(TCD.y_test[:100],TCD.y_test_hat[:100]))
np.sum(np.histogram(TCD.y_test_hat)[0])

24913

In [44]:
TCD.model.evaluate(TCD.X_test,TCD.y_test)

24608/24913 [============================>.] - ETA: 0s

[0.52790511442393295, 0.74992975554931163]